<a href="https://colab.research.google.com/github/devproviniciusalves/Crud-s-Vinny/blob/main/NF_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xml.etree.ElementTree as ET
import zipfile, os, io
from google.colab import files
import ipywidgets as widgets
from IPython.display import display

def extrair_numeros_nf(sequencia):
    if len(sequencia) % 44 != 0 or not sequencia.isdigit():
        return 'Sequência inválida'

    nfs = []
    for i in range(0, len(sequencia), 44):
        chave = sequencia[i:i+44]
        numero_nf = chave[27:34]
        nfs.append(numero_nf)

    return nfs

nfs_localizadas = []

chaves_nfs = input('Digite as chaves: ')
print('-' * 10)
print('Envie o arquivo ZIP com os XMLs:')
numeros_nf = extrair_numeros_nf(chaves_nfs)
if numeros_nf == 'Sequência inválida':
  print('Sequência inválida')
else:
  # print("Notas Bipadas")
  for numeros_nf in numeros_nf:
    # print(numeros_nf)
    nfs_localizadas.append(numeros_nf)

uploaded = files.upload()
print('-' * 10)

filename = list(uploaded.keys())[0]

ns = {'ns': 'http://www.portalfiscal.inf.br/nfe'}

with zipfile.ZipFile(io.BytesIO(uploaded[filename]), 'r') as zip_ref:
    numeros_nf = []

    for file_name in zip_ref.namelist():
        if file_name.endswith('.xml'):
            with zip_ref.open(file_name) as xml_file:
                try:
                    tree = ET.parse(xml_file)
                    root = tree.getroot()
                    # Buscar a tag <nNF> com namespace
                    nfe_num = root.find('.//ns:nNF', ns)
                    if nfe_num is not None:
                        numeros_nf.append(nfe_num.text)
                except Exception as e:
                    print(f"Erro ao ler {file_name}: {e}")

comparativo = list(set(numeros_nf) - set(nfs_localizadas))
conteudo_html = "<b>Notas Faltando</b><br>"
for nf in comparativo:
    conteudo_html += f"{nf}<br>"

caixa_resultado = widgets.HTML(
    value=f"""
    <div style='
        display: inline-block;
        text-align: center;
        padding: 10px;
        border: 1,5px solid #34516B;
        border-radius: 8px;
        background-color: #0D2A4A;
        color: white;
        min-width: 120px;
        max-width: 300px;
        min-height: 50px;
        max-height: 200px;
        overflow-y: auto;
    '>
        {conteudo_html}
    </div>
    """
)

display(caixa_resultado)

# !rm "{filename}"
!rm -r "{filename}"